In [3]:
import pandas as pd
import os
import string
from collections import Counter
import nltk
from natsort import natsorted
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
import PySimpleGUI as sg

This code is used to open existing RAW article and filter it. We are removing stop words, and splitting text into list of words.

In [4]:
def articleClean(file):
    stop_words = set((stopwords.words('english')) + list(string.punctuation))

    symbols = ["\n", "’", "“", "”", "'", "`", "-", "[", "]", "»", ":"] 

    article = ""
    
    for i in symbols:
        file = file.replace(i, " ")

    def listToString(s): #function that converts arr of strings into one string (used later)
        str1 = " "
        return (str1.join(s))
    
    word_tokens= word_tokenize(file.lower()) #converts arcivle into array of words

    filtered_article = [] 

    for w in word_tokens: # check if word from article is the extra , if not assign the word in the clean words array
        if w not in stop_words and w.isalpha() == True:
            filtered_article.append(w)

    article = listToString(filtered_article)

    return article

Creating lists of articles and genres.

In [5]:
all_articles = []
genres = []

Get all files as list

In [6]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/technology/training/")
tech_res_art = []
for i in range(1,101):
    tech_res = open("training_tech" + str(i) + ".txt", "r").read()
    tech_res = articleClean(tech_res)
    tech_res_art.append(tech_res)
    all_articles.append(tech_res)
    genres.append("technology")

#PRINT PASS

Get all sport files as list

In [7]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/sport/training/")
sport_res_art = []

for i in range(1,101):
    sport_res = open("training_sport" + str(i) + ".txt", "r").read()
    sport_res = articleClean(sport_res)
    sport_res_art.append(sport_res)
    all_articles.append(sport_res)
    genres.append("sport")

#PRINT PASS

Remove stopwords from all articles.

In [8]:
stop_words = stopwords.words('english')
word_tokens = word_tokenize("hello my name is Pierre")
filtered_data = []
print(word_tokens)
for word in word_tokens:
    if word not in stop_words:
        filtered_data.append(word)
print(filtered_data)

['hello', 'my', 'name', 'is', 'Pierre']
['hello', 'name', 'Pierre']


Create DataFrame.

In [9]:
d = {'articles' : all_articles, 'genres' : genres}
df = pd.DataFrame(data=d)
#df=df.transpose()
df

,articles,genres
0,european commission clarified claims perceived...,technology
1,audio player european union investigates micro...,technology
2,mentioned previously european commission embar...,technology
3,european commission responded reports perceive...,technology
4,god war ragnarok released days ago obviously s...,technology
...,...,...
195,dallas luka doncic admitted fatigue following ...,sport
196,portland trail blazers finished long six game ...,sport
197,nothing alleviates hangover nasty road trip li...,sport
198,arizona running back michael wiley right fends...,sport


Make everything lowercase.

In [10]:
df["articles"] = df["articles"].apply(lambda str : str.lower())
df

,articles,genres
0,european commission clarified claims perceived...,technology
1,audio player european union investigates micro...,technology
2,mentioned previously european commission embar...,technology
3,european commission responded reports perceive...,technology
4,god war ragnarok released days ago obviously s...,technology
...,...,...
195,dallas luka doncic admitted fatigue following ...,sport
196,portland trail blazers finished long six game ...,sport
197,nothing alleviates hangover nasty road trip li...,sport
198,arizona running back michael wiley right fends...,sport


Calculate the probability.

In [11]:
sum_s = 0
sum_t = 0
for i in df['genres']:
    if i == "sport":
        sum_s +=1
    else:
        sum_t +=1

prob_s = sum_s/len(df)
prob_t = sum_t/len(df)
print(prob_s)
print(prob_t)

0.5
0.5


Make a bag of words

In [12]:
df["bow"]  = df["articles"].str.split().apply(Counter)
df

,articles,genres,bow
0,european commission clarified claims perceived...,technology,"{'european': 4, 'commission': 7, 'clarified': ..."
1,audio player european union investigates micro...,technology,"{'audio': 1, 'player': 1, 'european': 1, 'unio..."
2,mentioned previously european commission embar...,technology,"{'mentioned': 1, 'previously': 1, 'european': ..."
3,european commission responded reports perceive...,technology,"{'european': 7, 'commission': 9, 'responded': ..."
4,god war ragnarok released days ago obviously s...,technology,"{'god': 4, 'war': 4, 'ragnarok': 4, 'released'..."
...,...,...,...
195,dallas luka doncic admitted fatigue following ...,sport,"{'dallas': 3, 'luka': 2, 'doncic': 12, 'admitt..."
196,portland trail blazers finished long six game ...,sport,"{'portland': 13, 'trail': 1, 'blazers': 12, 'f..."
197,nothing alleviates hangover nasty road trip li...,sport,"{'nothing': 1, 'alleviates': 1, 'hangover': 1,..."
198,arizona running back michael wiley right fends...,sport,"{'arizona': 9, 'running': 4, 'back': 4, 'micha..."


In [13]:
sum_bow_s = {}
sum_bow_t = {}
i=0
for bow in df["bow"]:
    for key in bow:
        if df["genres"][i] == "sport":
            if key in sum_bow_s:
                sum_bow_s[key] += bow[key]
            else:
                sum_bow_s[key] = bow[key]
        else:
            if key in sum_bow_t:
                sum_bow_t[key] +=bow[key]
            else:
                sum_bow_t[key] = bow[key]
    i += 1

#PRINT PASS

calculate total words/terms in Sport and Tech class.

In [14]:
term_occurences_s = 0
term_occurences_t = 0
for key in sum_bow_s:
    term_occurences_s += sum_bow_s[key]
for key in sum_bow_t:    
    term_occurences_t += sum_bow_t[key]

#print(term_occurences_t) 68593
#print(term_occurences_s) 64906

Calculate unique words.

In [15]:
temp = []
unique_term = 0
for bow in df["bow"]:
    for key in bow:
        if key not in temp:
            unique_term += 1
            temp.append(key)

#print(temp)
#print(unique_term) 19813

We store the probability of each word in training data in a dictionary called prob_bow

In [16]:
sum_bow = {}
term_occurences = 0
for bow in df["bow"]:
    for key in bow:
        if key in sum_bow:
            sum_bow[key] += bow[key]
        else:
            sum_bow[key] = bow[key]
        term_occurences += bow[key]

prob_bow = {}
# Adding +1 for better results
for key in sum_bow:
    prob_bow[key] = (sum_bow[key]/term_occurences)

prob_bow

{'european': 0.00017202593621807596,
 'commission': 0.00038375016540955403,
 'clarified': 3.969829297340214e-05,
 'claims': 0.00010586211459573905,
 'perceived': 5.2931057297869525e-05,
 'bias': 3.969829297340214e-05,
 'probe': 2.6465528648934763e-05,
 'microsoft': 0.0008733624454148472,
 'activision': 0.0003572846367606193,
 'blizzard': 7.939658594680428e-05,
 'deal': 0.0011512504962286623,
 'following': 0.000449913987031891,
 'comments': 0.00026465528648934763,
 'senior': 0.0002249569935159455,
 'staff': 0.0001984914648670107,
 'member': 7.939658594680428e-05,
 'social': 0.0001984914648670107,
 'media': 0.0002911208151382824,
 'ricardo': 6.616382162233691e-05,
 'cardoso': 0.0003308191081116845,
 'deputy': 7.939658594680428e-05,
 'head': 0.0010850866746063253,
 'unit': 0.0004102156940584888,
 'interinstitutional': 3.969829297340214e-05,
 'outreach': 6.616382162233691e-05,
 'governing': 1.3232764324467381e-05,
 'body': 0.0002117242291914781,
 'tweeted': 9.262935027127167e-05,
 'earlier

Probability of article being Sport or Technology related based on words in it.

In [ ]:
def split(words):
    return words.split()

def lower(words):
    return words.lower()

def probSport(article):
    article = split(lower(article))
    likelihood = 1
    evidence = 1
    for term in article:
        if term in sum_bow_s:
            likelihood *= (sum_bow_s[term]+1)/(term_occurences_s+unique_term)
        else:
            likelihood *= 1/(term_occurences_s+unique_term)
        evidence *= prob_bow[term]
    likelihood_prior = likelihood*prob_s
    posterior = likelihood_prior/evidence
    return posterior

def probTech(article):
    article = split(lower(article))
    likelihood = 1
    evidence = 1
    for term in article:
        if term in sum_bow_t:
            likelihood *= (sum_bow_t[term]+1)/(term_occurences_t+unique_term)
        else:
            likelihood *= 1/(term_occurences_t+unique_term)
        evidence *= prob_bow[term]
    likelihood_prior = likelihood*prob_t
    posterior = likelihood_prior/evidence
    return posterior

print(probSport(df["articles"][1]))
#print(df["articles"][1])
#print(probTech("technology")) 
#print(probSport("football player car team "))

creating Main method for UI.

In [ ]:
layout = [[sg.Text('Enter article number: ')],      
                 [sg.InputText()],      
                 [sg.Submit(), sg.Cancel()]]      


window = sg.Window('NaiveBaise predictor.', layout)    

event, values = window.read()    
window.close()

text_input = values[0]
final_output = ""
prob_sport = probSport(df["articles"][text_input])
prob_tech = probTech(df["articles"][text_input])
if prob_tech > prob_sport:
    final_output = "It is a tech article"
else: 
    final_output = "It is a sport article"
sg.popup(final_output)